# Ontologies
* An ontology is a collection different tools and classifications that can be used within a project's editor. Each tool or classification is called a "Feature Schema". 
* Feature Schemas contain information about the tool such as the kind, the name, all subclasses, and other information related to a tool. Feature Schemas can be shared between ontologies. 

* Helpful Links:
    * [Ontology documentation](https://docs.labelbox.com/docs/ontology-json)
    * [Project Setup Using Ontologies](https://github.com/Labelbox/labelbox-python/blob/develop/examples/project_configuration/project_setup.ipynb)

In [ ]:
!pip install labelbox

In [1]:
from labelbox import Client, OntologyBuilder, Tool, Classification, Option
import json

In [3]:
# Add your api key
API_KEY = None
ENDPOINT = "https://api.labelbox.com/graphql"
client = Client(api_key=API_KEY, endpoint=ENDPOINT)

### Create Ontology From Normalized Data
* Users can create ontologies from a json definition of the ontology
* See below `OntologyBuilder` section for more details on constructing the normalized ontology

In [4]:
# This will automatically create new feature schema
ontology_name = "sdk-ontology"
feature_schema_cat_normalized = {
     'tool': 'polygon', 
     'name': 'cat', 
     'color': 'black'
}

ontology_normalized_json = {"tools" : [feature_schema_cat_normalized], "classifications" : []}
ontology = client.create_ontology(name = ontology_name, normalized = ontology_normalized_json)
print(ontology)

<Ontology {'classification_schema_count': 0, 'created_at': datetime.datetime(2021, 11, 12, 21, 27, 32, 432000, tzinfo=datetime.timezone.utc), 'description': None, 'name': 'sdk-ontology', 'normalized': {'tools': [{'schemaNodeId': 'ckvww5f1302jl0z3o7cce0lmr', 'featureSchemaId': 'ckvww5f1302jk0z3o0f57fek3', 'required': False, 'name': 'cat', 'tool': 'polygon', 'color': 'black', 'classifications': []}], 'classifications': []}, 'object_schema_count': 1, 'uid': 'ckvww5f0q02jj0z3o7f7q3vk4', 'updated_at': datetime.datetime(2021, 11, 12, 21, 27, 32, 432000, tzinfo=datetime.timezone.utc)}>


### Create Ontology From Existing Feature Schemas
* It is often useful to support the same features in multiple ontologies. 
* Labelbox supports this workflow by allowing users to create ontologies using existing feature schemas.

In [5]:
# First create the feature schema
feature_schema_cat = client.create_feature_schema(feature_schema_cat_normalized)
# When we create the ontology it will not re-create the feature schema
print(feature_schema_cat.uid)
ontology = client.create_ontology_from_feature_schemas(ontology_name, [feature_schema_cat.uid])

ckvww5gjf02v60z5q4mcv5sf4


### Create Ontology From a Mix of New and Existing Feature Schemas
* If we want to create a new ontology that expands upon a previous ontology it is helpful to be able to share a portion of the features.
* To do this we will create the new schema ids that we want. Then we will create an ontology from the new list of ids.
* Note that for additional customization you can also combine the normalized json and use the create_ontology() method (not covered here).

In [6]:
# Create new dog schema id 
feature_schema_dog_normalized = {
     'tool': 'polygon', 
     'name': 'dog', 
     'color': 'black', 
     'classifications': [], 
}
feature_schema_dog = client.create_feature_schema(feature_schema_dog_normalized)
# The cat is shared between this new ontology and the one we created previously 
# (ie. the cat feature schema will not be re-created)
ontology = client.create_ontology_from_feature_schemas(ontology_name, [feature_schema_cat.uid, feature_schema_dog.uid])

### Read
* We can directly query by id for ontologies and feature schemas
* We also can search for both by name

In [7]:
#### Fetch by ID
feature_schema = client.get_feature_schema(feature_schema_cat.uid)
ontology = client.get_ontology(ontology.uid)
print(feature_schema)
print(ontology)

<FeatureSchema {'name': 'cat', 'normalized': {'schemaNodeId': 'ckvww5gjf02v70z5qd7650en8', 'featureSchemaId': 'ckvww5gjf02v60z5q4mcv5sf4', 'required': 0, 'name': 'cat', 'tool': 'polygon', 'color': 'black', 'classifications': []}, 'uid': 'ckvww5gjf02v60z5q4mcv5sf4'}>
<Ontology {'classification_schema_count': 0, 'created_at': datetime.datetime(2021, 11, 12, 21, 27, 36, 767000, tzinfo=datetime.timezone.utc), 'description': None, 'name': 'sdk-ontology', 'normalized': {'tools': [{'schemaNodeId': 'ckvww5gjf02v70z5qd7650en8', 'featureSchemaId': 'ckvww5gjf02v60z5q4mcv5sf4', 'required': False, 'name': 'cat', 'tool': 'polygon', 'color': 'black', 'classifications': []}, {'schemaNodeId': 'ckvww5hke02f70z5cdwzwccvw', 'featureSchemaId': 'ckvww5hke02f60z5c8yb1cb0t', 'required': False, 'name': 'dog', 'tool': 'polygon', 'color': 'black', 'classifications': []}], 'classifications': []}, 'object_schema_count': 0, 'uid': 'ckvww5id802ki0z3o166v6sz9', 'updated_at': datetime.datetime(2021, 11, 12, 21, 27, 36

In [8]:
#### Search by name
feature_schema = next(client.get_feature_schemas("cat"))
ontology = next(client.get_ontologies(ontology_name))
print(feature_schema)
print(ontology)

<FeatureSchema {'name': 'cat', 'normalized': {'schemaNodeId': 'ckvww5gjf02v70z5qd7650en8', 'featureSchemaId': 'ckvww5gjf02v60z5q4mcv5sf4', 'required': 0, 'name': 'cat', 'tool': 'polygon', 'color': 'black', 'classifications': []}, 'uid': 'ckvww5gjf02v60z5q4mcv5sf4'}>
<Ontology {'classification_schema_count': 0, 'created_at': datetime.datetime(2021, 11, 12, 21, 27, 36, 767000, tzinfo=datetime.timezone.utc), 'description': None, 'name': 'sdk-ontology', 'normalized': {'tools': [{'schemaNodeId': 'ckvww5gjf02v70z5qd7650en8', 'featureSchemaId': 'ckvww5gjf02v60z5q4mcv5sf4', 'required': False, 'name': 'cat', 'tool': 'polygon', 'color': 'black', 'classifications': []}, {'schemaNodeId': 'ckvww5hke02f70z5cdwzwccvw', 'featureSchemaId': 'ckvww5hke02f60z5c8yb1cb0t', 'required': False, 'name': 'dog', 'tool': 'polygon', 'color': 'black', 'classifications': []}], 'classifications': []}, 'object_schema_count': 0, 'uid': 'ckvww5id802ki0z3o166v6sz9', 'updated_at': datetime.datetime(2021, 11, 12, 21, 27, 36

### Update and Delete
- At this time, these options are not supported from the SDK.
- Updating an ontology is dangerous and could cause labels to be hidden. 
    - Use caution when doing so

### Ontology Builder
* The ontology builder is a tool for creating and modifying normalized json

In [9]:
# Create normalized json with a bounding box and segmentation tool
ontology_builder = OntologyBuilder(tools=[
    Tool(tool=Tool.Type.BBOX, name="dog"),
    Tool(tool=Tool.Type.SEGMENTATION, name="cat"),
])
# Creating an ontology from this is easy
ontology = client.create_ontology("ontology-builder-ontology", ontology_builder.asdict())
print(json.dumps(ontology.normalized, indent = 2))

{
  "tools": [
    {
      "schemaNodeId": "ckvww5ka002g90z5c4d1m1qtr",
      "featureSchemaId": "ckvww5ka002g80z5c7skh9w9i",
      "required": false,
      "name": "dog",
      "tool": "rectangle",
      "color": "#ff0000",
      "classifications": []
    },
    {
      "schemaNodeId": "ckvww5ka002gb0z5cbw1rb4rf",
      "featureSchemaId": "ckvww5ka002ga0z5ccpj953nf",
      "required": false,
      "name": "cat",
      "tool": "superpixel",
      "color": "#00ffff",
      "classifications": []
    }
  ],
  "classifications": []
}


* Alternative syntax for defining the ontology via the OntologyBuilder

In [10]:
# Create
ontology_builder = OntologyBuilder()
# Append tools
tool_dog = Tool(tool=Tool.Type.BBOX, name="dog")
tool_cat = Tool(tool=Tool.Type.SEGMENTATION, name="cat")
ontology_builder.add_tool(tool_dog)
ontology_builder.add_tool(tool_cat)
ontology = client.create_ontology("ontology-builder-ontology", ontology_builder.asdict())
print(json.dumps(ontology.normalized, indent = 2))

{
  "tools": [
    {
      "schemaNodeId": "ckvww5kl102lc0z3ogyps5e91",
      "featureSchemaId": "ckvww5kl102lb0z3o6oce4878",
      "required": false,
      "name": "dog",
      "tool": "rectangle",
      "color": "#ff0000",
      "classifications": []
    },
    {
      "schemaNodeId": "ckvww5kl202le0z3oeq6b8kzv",
      "featureSchemaId": "ckvww5kl202ld0z3o6i7c33r8",
      "required": false,
      "name": "cat",
      "tool": "superpixel",
      "color": "#00ffff",
      "classifications": []
    }
  ],
  "classifications": []
}


* Classifications are supported too (Both for top level and as subclassifications)

In [11]:
 
ontology_builder = OntologyBuilder(
    tools=[
        Tool(tool=Tool.Type.BBOX, name="dog"),
        Tool(tool=Tool.Type.SEGMENTATION,
             name="cat",
             classifications=[
                 Classification(class_type=Classification.Type.TEXT,
                                instructions="name")
             ])
    ],
    classifications=[
        Classification(class_type=Classification.Type.RADIO,
                       instructions="image_quality",
                       options=[Option(value="clear"),
                                Option(value="blurry")])
    ])
print(json.dumps(ontology_builder.asdict(), indent = 2))

{
  "tools": [
    {
      "tool": "rectangle",
      "name": "dog",
      "required": false,
      "color": "#ff0000",
      "classifications": [],
      "schemaNodeId": null,
      "featureSchemaId": null
    },
    {
      "tool": "superpixel",
      "name": "cat",
      "required": false,
      "color": "#00ffff",
      "classifications": [
        {
          "type": "text",
          "instructions": "name",
          "name": "name",
          "required": false,
          "options": [],
          "schemaNodeId": null,
          "featureSchemaId": null
        }
      ],
      "schemaNodeId": null,
      "featureSchemaId": null
    }
  ],
  "classifications": [
    {
      "type": "radio",
      "instructions": "image_quality",
      "name": "image_quality",
      "required": false,
      "options": [
        {
          "schemaNodeId": null,
          "featureSchemaId": null,
          "label": "clear",
          "value": "clear",
          "options": []
        },
        {
     

* All Tool objects are constructed the same way:

In [12]:
bbox_tool = Tool(tool=Tool.Type.BBOX, name="dog_box")
poly_tool = Tool(tool=Tool.Type.POLYGON, name="dog_poly")
seg_tool = Tool(tool=Tool.Type.SEGMENTATION, name="dog_seg")
point_tool = Tool(tool=Tool.Type.POINT, name="dog_center")
line_tool = Tool(tool=Tool.Type.LINE, name="dog_orientation")
ner_tool = Tool(tool=Tool.Type.NER, name="dog_reference")

* Classifications are all constructed the same way (except text which doesn't require options)
* Classifications can be global or subclasses to a tool (ie dog bounding box, with a breed classification)

In [13]:

text_classification = Classification(class_type=Classification.Type.TEXT,
                                     instructions="dog_name")
radio_classification = Classification(class_type=Classification.Type.CHECKLIST,
                                      instructions="dog_breed",
                                      options=[Option("poodle")])
dropdown_classification = Classification(
    class_type=Classification.Type.DROPDOWN,
    instructions="dog_features",
    options=[Option("short"), Option("fluffy")])
checklist_classification = Classification(
    class_type=Classification.Type.CHECKLIST,
    instructions="background",
    options=[Option("at_park"), Option("has_leash")])